In [9]:
# Import required functions and classes
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image, read_image_as_pil
from sahi.utils.file import Path, increment_path, list_files, save_json, save_pickle, download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict, agg_prediction, get_prediction_batched, get_sliced_prediction_batched, predict 
from sahi.prediction import visualize_object_predictions
from IPython.display import Image
from numpy import asarray
import cv2
import os
import time
from PIL import Image
import json
import numpy as np
from tqdm import tqdm
import time
from sahi.prediction import ObjectPrediction, PredictionResult
from pathlib import Path

In [10]:
# Download YOLOv8-S model to 'models/yolov8s.pt'
yolov8_model_path = 'models/yolov8/last.pt'
#download_yolov8s_model(destination_path=yolov8_model_path)

In [11]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=yolov8_model_path,
    confidence_threshold=0.3,
    device="cuda:0", # or 'cpu'
)

In [12]:
def get_slice_parameters(object_density, slice_size):
    
    #start_time = time.time()
    #image_path = "test_data/0000006_06773_d_0000018.jpg"
    #image = Image.open(image_path).convert("RGB")
    #image_width, image_height  = image.size
    #print("Image Width:", image_width)
    #print("Image Height:", image_height)
    #min_dim = min(image_width, image_height)
    #slice_size = min_dim // 4 if min_dim > 1600 else min_dim // 2
    #print(f"Dimension calculation time taken: {(time.time() - start_time)*1000:.2f} ms")

    
    if object_density >= 50:
        #slice_size = min_dim // 4
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.5
        overlap_height_ratio = 0.5
    elif 25 <= object_density < 50:
        #slice_size = min_dim // 2
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.25
        overlap_height_ratio = 0.25
    elif 10 <= object_density < 25:
        #slice_size = min_dim // 2
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.15
        overlap_height_ratio = 0.15
    else:
        return

    return slice_width, slice_height, overlap_width_ratio, overlap_height_ratio

In [13]:
# Function to get image details by image_id
def get_image_id(coco_data, image_name):
    for image in coco_data["images"]:
        file_name = Path(image['file_name']).stem
        if file_name == image_name:
            return image['id']
    return None

# export visualization
def predict_sliced_images(input_folder, dataset_json_path, detection_model, slice_size):
    """
    Processes all image files in input_folder:
      - Runs predictions using get_prediction function and detection_model.
      - Saves annotated images with bounding boxes in output_folder.
      - Saves prediction details as JSON files in output_folder.
    
    Parameters:
      input_folder (str): Path to the folder containing images.
      detection_model: Your detection model used for prediction.
      slice_size (int): Parameter for slice size used in get_slice_parameters.
    """
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)
    
    # Visualization parameters
    visual_bbox_gt_thickness = 3
    visual_bbox_thickness = 2
    visual_text_size = 0.5
    visual_text_thickness = 1
    visual_hide_labels = False
    visual_hide_conf = False
    visual_export_format = 'png'
    
    sliced_predictions = []
    coco_json = []
    
    # Initialize a variable to accumulate total prediction time for all images.
    total_prediction_time = 0.0
    
    # Loop over files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_folder, filename)
            image_as_pil = read_image_as_pil(image_path)
            filename_without_ext = Path(filename).stem
            
            print("*****************************************")
            print("File Name", filename_without_ext)

            img_id = get_image_id(data, filename_without_ext)
            
            # Get initial predictions from your detection model
            time_start = time.time()
            prediction = get_prediction(image_path, detection_model)
            time_end = time.time() - time_start
            print("Initial Prediction time is: {:.2f} ms".format(time_end * 1000))
            
            # Add initial prediction time to the cumulative total.
            iteration_time = time_end
            
            object_density = len(prediction.object_prediction_list)
            print("Object Density:", object_density)
            
            if object_density > 10:
                slice_width, slice_height, overlap_width_ratio, overlap_height_ratio = get_slice_parameters(object_density, slice_size)
    
                print("********* Slice Parameters ***********")
                print("Slice Width: ", slice_width)
                print("Slice Height: ", slice_height)
                print("Overlap Width Ratio: ", overlap_width_ratio)
                print("Overlap Height Ratio: ", overlap_height_ratio)

                time_start_slice = time.time()
                result_sahi = get_sliced_prediction(
                    image_path,
                    detection_model,
                    slice_height=slice_height,
                    slice_width=slice_width,
                    overlap_height_ratio=overlap_height_ratio,
                    overlap_width_ratio=overlap_width_ratio,
                    postprocess_min_area=16,
                    postprocess_type="OptNMS",
                    verbose=2
                )
                time_end_slice = time.time() - time_start_slice
                print("Sliced Prediction time is: {:.2f} ms".format(time_end_slice * 1000))
                
                # Add sliced prediction time to the current iteration's total.
                iteration_time += time_end_slice
                
                coco_prediction = result_sahi.to_coco_predictions(image_id=img_id)
                for idx, predict in enumerate(coco_prediction):
                    if coco_prediction[idx]["bbox"]:
                        coco_json.append(predict)
                    
                sliced_predictions.append(result_sahi)
                
                visualize_object_predictions(
                    np.ascontiguousarray(image_as_pil),
                    object_prediction_list=result_sahi.object_prediction_list,
                    rect_th=visual_bbox_thickness,
                    text_size=visual_text_size,
                    text_th=visual_text_thickness,
                    hide_labels=visual_hide_labels,
                    hide_conf=visual_hide_conf,
                    output_dir=save_dir,
                    file_name=filename_without_ext,
                    export_format=visual_export_format,
                )
                
            else:
                print("Prediction time is: {:.2f} ms".format(time_end * 1000))
                
                coco_prediction = prediction.to_coco_predictions(image_id=img_id)
                for idx, predict in enumerate(coco_prediction):
                    if coco_prediction[idx]["bbox"]:
                        coco_json.append(predict)
                    
                sliced_predictions.append(prediction)
                
                visualize_object_predictions(
                    np.ascontiguousarray(image_as_pil),
                    object_prediction_list=prediction.object_prediction_list,
                    rect_th=visual_bbox_thickness,
                    text_size=visual_text_size,
                    text_th=visual_text_thickness,
                    hide_labels=visual_hide_labels,
                    hide_conf=visual_hide_conf,
                    output_dir=save_dir,
                    file_name=filename_without_ext,
                    export_format=visual_export_format,
                )
            
            # Update the overall total prediction time
            total_prediction_time += iteration_time
                 
    if dataset_json_path:
        save_path = str(save_dir / "result.json")
        save_json(coco_json, save_path)
        print(f"Prediction results are successfully exported to {save_dir}")
    
    print(f"Prediction Completed Successfully: {len(sliced_predictions)} images")
    print("Total Prediction time for all images is: {:.2f} ms".format(total_prediction_time * 1000))
    return sliced_predictions


#### **Single Images**

In [29]:
# Example usage: Adaptive-Optimized-NMS (NMS)
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 512
result_preds_adapt_opt_nms= predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000011_05068_d_0000008
Initial Prediction time is: 22.60 ms
Object Density: 33
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 8
POST PROCESS:  NMS
Performing prediction on 8 slices.
Original Prediction Count 93
Final Bounding Box Count (NMS):  31
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  6
Slicing performed in 0.00536036491394043 seconds.
Prediction performed in 0.15665888786315918 seconds.
Sliced Prediction time is: 161.47 ms
Prediction results are successfully exported to sliced_predictions/exp233
Prediction Completed Successfully: 1 images
Total Prediction time for all images is: 184.08 ms


In [7]:
# Example usage: Adaptive-Optimized-NMS (OptNMS-Change)
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 512
result_preds_adapt_opt_nms= predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000011_05068_d_0000008


/mmfs1/cm/shared/apps_local/python/3.11/envs/torch11.8/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initial Prediction time is: 994.80 ms
Object Density: 33
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 8
POST PROCESS:  OptNMS
Performing prediction on 8 slices.
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  63
Final Bounding Box Count (OptNMS): 33
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 6
Slicing performed in 0.006620883941650391 seconds.
Prediction performed in 0.16126465797424316 seconds.
Sliced Prediction time is: 166.81 ms
Prediction results are successfully exported to sli

In [20]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 512
result_preds_adapt_opt_nms_iou_only_area = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000011_05068_d_0000008
Initial Prediction time is: 84.30 ms
Object Density: 33
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 8
POST PROCESS:  OptNMS
Performing prediction on 8 slices.
Original Prediction Count 93
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  63
Final Bounding Box Count (OptNMS): 33
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 6
Slicing performed in 0.0065076351165771484 seconds.
Prediction perf

In [8]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './sliced_predictions/exp233/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.082
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.149
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.068
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.096
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.429
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.046
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.265
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [10]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './sliced_predictions/exp165/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.00s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.073
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.122
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.086
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.047
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.177
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [21]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './sliced_predictions/exp161/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.082
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.148
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.067
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.096
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.429
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.046
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.265
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [31]:
result_predict_nms = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 50
POST PROCESS:  NMS
Performing prediction on 50 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  1.62it/s]

Original Prediction Count 271
Final Bounding Box Count (NMS):  44
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  30
Final Bounding Box Count (NMS):  22
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  11
Prediction time is: 499.46 ms
Prediction results are successfully exported to runs/predict/exp232
Model loaded in 0.19486474990844727 seconds.
Slicing performed in 0.0012063980102539062 seconds.
Prediction performed in 0.4994645118713379 seconds.
Exporting performed in 0.048632144927978516 seconds.


In [24]:
result_predict_nms = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  NMS
Performing prediction on 10 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  2.94it/s]

Original Prediction Count 148
Final Bounding Box Count (NMS):  34
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  10
Prediction time is: 233.49 ms
Prediction results are successfully exported to runs/predict/exp230
Model loaded in 0.09711647033691406 seconds.
Slicing performed in 0.001035451889038086 seconds.
Prediction performed in 0.233489990234375 seconds.
Exporting performed in 0.04587388038635254 seconds.


In [25]:
result_predict_nms = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  OptNMS
Performing prediction on 10 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  2.94it/s]

Original Prediction Count 148
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  68
Final Bounding Box Count (OptNMS): 32
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 7
Prediction time is: 232.44 ms
Prediction results are successfully exported to runs/predict/exp231
Model loaded in 0.02906179428100586 seconds.
Slicing performed in 0.0010347366333007812 seconds.
Prediction performed in 0.23243927955627441 seconds.
Exporting performed in 0.04718732833862305 seconds.


In [14]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './runs/predict/exp209/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.059
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.110
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.050
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.062
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.545
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.029
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.250
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [15]:
result_predict_optnms_less_conf1 = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         postprocess_min_area = 576,
                         postprocess_conf_threshold = 0.2,                  
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  OptNMS
Performing prediction on 10 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  2.94it/s]

Original Prediction Count 148
Intial Min Area:  576
Confidence Scores:  0.2
Min area Threshold:  576
Adaptive Filtered Prediction:  55
Final Bounding Box Count (OptNMS): 24
Adaptive Filtered Prediction:  12
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  15
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 9
Prediction time is: 233.04 ms
Prediction results are successfully exported to runs/predict/exp202
Model loaded in 0.30477356910705566 seconds.
Slicing performed in 0.0011668205261230469 seconds.
Prediction performed in 0.23304462432861328 seconds.
Exporting performed in 0.04672050476074219 seconds.


In [13]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './runs/predict/exp200/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.059
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.110
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.050
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.062
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.545
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.029
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.250
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [14]:
result_predict_optnms_less_nms = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 576,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  NMS
Performing prediction on 10 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  2.91it/s]

Original Prediction Count 148
Final Bounding Box Count (NMS):  34
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  10
Prediction time is: 235.37 ms
Prediction results are successfully exported to runs/predict/exp201
Model loaded in 0.3096604347229004 seconds.
Slicing performed in 0.0013873577117919922 seconds.
Prediction performed in 0.23536896705627441 seconds.
Exporting performed in 0.04634213447570801 seconds.


In [9]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './runs/predict/exp198/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.068
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.136
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.078
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.066
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [10]:
result_predict_optnms_less_nms = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 900,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  NMS
Performing prediction on 10 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  2.92it/s]

Original Prediction Count 148
Final Bounding Box Count (NMS):  34
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  10
Prediction time is: 235.30 ms
Prediction results are successfully exported to runs/predict/exp199
Model loaded in 0.3071262836456299 seconds.
Slicing performed in 0.0013284683227539062 seconds.
Prediction performed in 0.23529839515686035 seconds.
Exporting performed in 0.046747446060180664 seconds.


In [11]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './runs/predict/exp199/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.084
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.429
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [5]:
result_predict_optnms_less_conf1 = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         postprocess_min_area =  1024,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  OptNMS
Performing prediction on 10 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  2.92it/s]

Original Prediction Count 148
Intial Min Area:  1024
Confidence Scores:  0.3
Min area Threshold:  1024
Adaptive Filtered Prediction:  14
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  14
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 9
Prediction time is: 232.51 ms
Prediction results are successfully exported to runs/predict/exp196
Model loaded in 0.034358978271484375 seconds.
Slicing performed in 0.0011708736419677734 seconds.
Prediction performed in 0.23250913619995117 seconds.
Exporting performed in 0.049688100814819336 seconds.


In [6]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './runs/predict/exp196/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.073
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.147
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.086
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.076
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [7]:
result_predict_optnms_less_conf1 = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  NMS
Performing prediction on 10 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  2.92it/s]

Original Prediction Count 148
Final Bounding Box Count (NMS):  34
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  10
Prediction time is: 235.40 ms
Prediction results are successfully exported to runs/predict/exp191
Model loaded in 0.030333518981933594 seconds.
Slicing performed in 0.0011222362518310547 seconds.
Prediction performed in 0.2353966236114502 seconds.
Exporting performed in 0.04644370079040527 seconds.


In [9]:
result_predict_optnms_less_conf = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  OptNMS
Performing prediction on 10 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  2.94it/s]

Original Prediction Count 148
Adaptive Filtered Prediction:  88
Final Bounding Box Count (OptNMS): 35
Adaptive Filtered Prediction:  18
Final Bounding Box Count (OptNMS): 9
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  15
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  15
Final Bounding Box Count (OptNMS): 10
Prediction time is: 232.35 ms
Prediction results are successfully exported to runs/predict/exp188
Model loaded in 0.13028502464294434 seconds.
Slicing performed in 0.0010516643524169922 seconds.
Prediction performed in 0.2323460578918457 seconds.
Exporting performed in 0.04683375358581543 seconds.


In [8]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './runs/predict/exp187/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.084
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.429
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [7]:
result_predict_optnms_only_area = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  OptNMS
Performing prediction on 10 slices.


/mmfs1/cm/shared/apps_local/python/3.11/envs/torch11.8/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Performing inference on images: 100%|██████████| 1/1 [00:01<00:00,  1.27s/it]

Original Prediction Count 148
Adaptive Filtered Prediction:  88
Final Bounding Box Count (OptNMS): 35
Adaptive Filtered Prediction:  18
Final Bounding Box Count (OptNMS): 9
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  15
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  15
Final Bounding Box Count (OptNMS): 10
Prediction time is: 1156.84 ms
Prediction results are successfully exported to runs/predict/exp186
Model loaded in 0.03148341178894043 seconds.
Slicing performed in 0.0016107559204101562 seconds.
Prediction performed in 1.1568357944488525 seconds.
Exporting performed in 0.04780077934265137 seconds.


In [8]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './runs/predict/exp186/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.084
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.429
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [9]:
result_predict_optnms_10 = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  OptNMS
Performing prediction on 10 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  2.94it/s]

Original Prediction Count 148
Adaptive Filtered Prediction:  68
Final Bounding Box Count (OptNMS): 32
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 7
Prediction time is: 232.92 ms
Prediction results are successfully exported to runs/predict/exp184
Model loaded in 0.034542083740234375 seconds.
Slicing performed in 0.0011737346649169922 seconds.
Prediction performed in 0.23291969299316406 seconds.
Exporting performed in 0.047029972076416016 seconds.


In [11]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './runs/predict/exp184/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.070
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.121
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.084
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.069
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.483
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.040
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.255
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [10]:
result_predict_nms_10 = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  NMS
Performing prediction on 10 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  2.93it/s]

Final Bounding Box Count (NMS):  34
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  10
Prediction time is: 235.10 ms
Prediction results are successfully exported to runs/predict/exp185
Model loaded in 0.029547929763793945 seconds.
Slicing performed in 0.0011701583862304688 seconds.
Prediction performed in 0.23509502410888672 seconds.
Exporting performed in 0.0463709831237793 seconds.


In [12]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './runs/predict/exp185/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.084
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.429
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [11]:
result_predict_optnms_less_only_area = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         postprocess_min_area =  1024,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  OptNMS
Performing prediction on 10 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  2.97it/s]

Original Prediction Count 148
Intial Min Area:  1024
Confidence Scores:  0.3
Min area Threshold:  1024
Adaptive Filtered Prediction:  14
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  14
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 9
Prediction time is: 232.52 ms
Prediction results are successfully exported to runs/predict/exp208
Model loaded in 0.060709238052368164 seconds.
Slicing performed in 0.0010657310485839844 seconds.
Prediction performed in 0.23251914978027344 seconds.
Exporting performed in 0.046645402908325195 seconds.


In [12]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './runs/predict/exp208/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.073
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.147
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.086
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.076
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [7]:
result_predict_nms_11 = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  NMS
Performing prediction on 10 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  2.90it/s]

Original Prediction Count 148
Final Bounding Box Count (NMS):  34
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  10
Prediction time is: 237.13 ms
Prediction results are successfully exported to runs/predict/exp207
Model loaded in 0.09213733673095703 seconds.
Slicing performed in 0.0010538101196289062 seconds.
Prediction performed in 0.23713278770446777 seconds.
Exporting performed in 0.04656219482421875 seconds.


In [8]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './runs/predict/exp207/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.143
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.084
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.429
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

##### **Subset of 10 images**

In [16]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 128
result_preds_adapt_nms_no_adaptive_iou = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000074_03738_d_0000007
Initial Prediction time is: 98.93 ms
Object Density: 52
********* Slice Parameters ***********
Slice Width:  128
Slice Height:  128
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 464
POST PROCESS:  OptNMS
Performing prediction on 464 slices.
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  64
Final Bounding Box Count (OptNMS): 49
Adaptive Filtered Prediction:  183
Final Bounding Box Count (OptNMS): 81
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  31
Final Bounding Box Count (OptNMS): 16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Slicing performed in 0.011892318725585938 seconds.
Prediction performed in 2.946497678756714 seconds.
Sliced Prediction time is: 2954.62 ms
****************************************

In [17]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp327/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.44s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.032
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.056
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.032
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.076
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.960
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [18]:
result_predict_optnms = predict(source='./test_visdrone_data',
                         dataset_json_path = './subset_visdrone_test_data_10.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.5,                  
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Image Name: 0000074_03738_d_0000007
Image Size:  (1920, 1080)
Sliced Boxes Count: 112
POST PROCESS:  OptNMS
Performing prediction on 112 slices.


Performing inference on images:  10%|█         | 1/10 [00:01<00:10,  1.19s/it]

Confidence Scores:  0.5
Min area Threshold:  16
Adaptive Filtered Prediction:  22
Final Bounding Box Count (OptNMS): 22
Adaptive Filtered Prediction:  49
Final Bounding Box Count (OptNMS): 27
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 3
Prediction time is: 993.35 ms


...

Confidence Scores:  0.5
Min area Threshold:  16
Adaptive Filtered Prediction:  42
Final Bounding Box Count (OptNMS): 21
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Prediction time is: 947.69 ms
Image Name: 0000074_01218_d_0000002
Image Size:  (1920, 1080)
Sliced Boxes Count: 112
POST PROCESS:  OptNMS
Performing prediction on 112 slices.


Performing inference on images:  90%|█████████ | 9/10 [00:10<00:01,  1.14s/it]

Confidence Scores:  0.5
Min area Threshold:  16
Adaptive Filtered Prediction:  61
Final Bounding Box Count (OptNMS): 40
Adaptive Filtered Prediction:  44
Final Bounding Box Count (OptNMS): 17
Adaptive Filtered Prediction:  29
Final Bounding Box Count (OptNMS): 11
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 2
Prediction time is: 944.60 ms
Image Name: 0000074_08777_d_0000017
Image Size:  (1920, 1080)
Sliced Boxes Count: 112
POST PROCESS:  OptNMS
Performing prediction on 112 slices.


Performing inference on images: 100%|██████████| 10/10 [00:11<00:00,  1.15s/it]

Confidence Scores:  0.5
Min area Threshold:  16
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  74
Final Bounding Box Count (OptNMS): 24
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 8
Prediction time is: 935.99 ms
Prediction results are successfully exported to runs/predict/exp243
Model loaded in 0.30088090896606445 seconds.
Slicing performed in 0.025902509689331055 seconds.
Prediction performed in 9.594115495681763 seconds.
Exporting performed in 0.07857441902160645 seconds.


In [19]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp243/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.22s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.001
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.005
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [7]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 512
result_preds_adapt_nms_no_adaptive_iou = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000074_03738_d_0000007


/mmfs1/cm/shared/apps_local/python/3.11/envs/torch11.8/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initial Prediction time is: 1027.06 ms
Object Density: 52
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  TruncatedNMS
Performing prediction on 28 slices.
Original Prediction Count 408
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  408
Final Bounding Box Count (Truncated NMS):  96
Slicing performed in 0.011307001113891602 seconds.
Prediction performed in 0.7458071708679199 seconds.
Sliced Prediction time is: 755.58 ms
*****************************************
File Name 0000074_07850_d_0000015
Initial Prediction time is: 38.02 ms
Object Density: 53
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  TruncatedNMS
Performing prediction on 28 slices.
Original Prediction Count 826
Confidence Scores:  0.3
Min area Threshold:  16
A

In [8]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-TruncatedNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp209/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.86s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.147
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.208
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.155
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.183
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.118
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.672
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.058
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [8]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 512
result_preds_adapt_nms_no_adaptive_iou = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000074_03738_d_0000007
Initial Prediction time is: 37.87 ms
Object Density: 52
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  TruncatedNMS
Performing prediction on 28 slices.
Original Prediction Count 408
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  408
Final Bounding Box Count (Truncated NMS):  406
Slicing performed in 0.010332822799682617 seconds.
Prediction performed in 0.5204071998596191 seconds.
Sliced Prediction time is: 528.91 ms
*****************************************
File Name 0000074_07850_d_0000015
Initial Prediction time is: 38.52 ms
Object Density: 53
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  TruncatedNMS
Performing prediction on 28 slices.
Orig

In [9]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-TruncatedNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp206/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.00s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.141
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.181
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.159
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.116
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.066
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.961
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.068
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [8]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 512
result_preds_adapt_nms_no_adaptive_iou = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000074_03738_d_0000007
Initial Prediction time is: 37.86 ms
Object Density: 52
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.
Original Prediction Count 408
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  382
Final Bounding Box Count (OptNMS): 62
Adaptive Filtered Prediction:  12
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Slicing performed in 0.0104522705078125 seconds.
Prediction performed in 0.4811079502105713 seconds.
Sliced Prediction time is: 489.89 ms
*****************************************
File Name 0000074_07850_

In [9]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp198/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.71s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.158
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.220
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.170
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.117
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.921
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.058
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [8]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 512
result_preds_adapt_nms_conf_only = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000074_03738_d_0000007
Initial Prediction time is: 36.95 ms
Object Density: 52
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.
Original Prediction Count 408
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  382
Final Bounding Box Count (OptNMS): 81
Adaptive Filtered Prediction:  12
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Slicing performed in 0.010486125946044922 seconds.
Prediction performed in 0.47877025604248047 seconds.
Sliced Prediction time is: 486.78 ms
*****************************************
File Name 0000074_078

In [ ]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp195/result.json'

In [10]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp195/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.87s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.160
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.225
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.171
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.184
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.123
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.938
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.060
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [9]:
result_predict_optnms_10_no_iou_vary = predict(source='./test_visdrone_data',
                         dataset_json_path = './subset_visdrone_test_data_10.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         postprocess_min_area =  16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Image Name: 0000074_03738_d_0000007
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Original Prediction Count 548
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  382
Final Bounding Box Count (OptNMS): 62
Adaptive Filtered Prediction:  12
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Prediction time is: 528.54 ms


...

Original Prediction Count 638
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  408
Final Bounding Box Count (OptNMS): 84
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  14
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  43
Final Bounding Box Count (OptNMS): 15
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 3
Prediction time is: 475.43 ms


Performing inference on images:  90%|█████████ | 9/10 [00:06<00:00,  1.43it/s]

Image Name: 0000074_08777_d_0000017
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.


Performing inference on images: 100%|██████████| 10/10 [00:06<00:00,  1.43it/s]

Original Prediction Count 174
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  26
Final Bounding Box Count (OptNMS): 22
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  60
Final Bounding Box Count (OptNMS): 20
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  19
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 2
Prediction time is: 456.42 ms
Prediction results are successfully exported to runs/predict/exp222
Model loaded in 0.03627920150756836 seconds.
Slicing performed in 0.023417949676513672 seconds.
Prediction performed in 4.920410871505737 seconds.
Exporting performed in 0.07914948463439941 seconds.


In [10]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp222/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.78s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.024
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.055
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.019
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.058
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.010
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.074
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.020
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [8]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp221/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.78s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.024
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.055
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.019
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.058
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.010
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.074
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.020
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [11]:
result_predict_optnms_10_conf = predict(source='./test_visdrone_data',
                         dataset_json_path = './subset_visdrone_test_data_10.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         postprocess_min_area =  16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Image Name: 0000074_03738_d_0000007
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Original Prediction Count 548
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  382
Final Bounding Box Count (OptNMS): 81
Adaptive Filtered Prediction:  12
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Prediction time is: 522.00 ms


...

Original Prediction Count 638
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  408
Final Bounding Box Count (OptNMS): 101
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  14
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  43
Final Bounding Box Count (OptNMS): 15
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 3
Prediction time is: 464.26 ms


Performing inference on images:  90%|█████████ | 9/10 [00:06<00:00,  1.44it/s]

Image Name: 0000074_08777_d_0000017
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.


Performing inference on images: 100%|██████████| 10/10 [00:06<00:00,  1.43it/s]

Original Prediction Count 174
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  26
Final Bounding Box Count (OptNMS): 22
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  60
Final Bounding Box Count (OptNMS): 22
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  19
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 2
Prediction time is: 444.48 ms
Prediction results are successfully exported to runs/predict/exp220
Model loaded in 0.30907201766967773 seconds.
Slicing performed in 0.024590015411376953 seconds.
Prediction performed in 4.7903947830200195 seconds.
Exporting performed in 0.07849001884460449 seconds.


In [12]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp220/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.94s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.025
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.056
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.020
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.056
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.010
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.078
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.022
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [12]:
# Example usage: Adaptive-Optimized-NMS (NMM)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 512
result_preds_adapt_greedynmm_iou = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000074_03738_d_0000007
Initial Prediction time is: 41.63 ms
Object Density: 52
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  NMM
Performing prediction on 28 slices.
Slicing performed in 0.01087498664855957 seconds.
Prediction performed in 0.5793893337249756 seconds.
Sliced Prediction time is: 589.03 ms
*****************************************
File Name 0000074_07850_d_0000015
Initial Prediction time is: 37.79 ms
Object Density: 53
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  NMM
Performing prediction on 28 slices.
Slicing performed in 0.010836124420166016 seconds.
Prediction performed in 0.5948703289031982 seconds.
Sliced Prediction time is: 603.96 ms
*****************************************
F

In [13]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-NMM)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp191/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.75s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.216
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.118
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.931
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.071
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.052
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [9]:
# Example usage: Adaptive-Optimized-NMS (GreedyNMM)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 512
result_preds_adapt_greedynmm_iou = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000074_03738_d_0000007
Initial Prediction time is: 36.32 ms
Object Density: 52
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  GREEDYNMM
Performing prediction on 28 slices.
Slicing performed in 0.010018110275268555 seconds.
Prediction performed in 0.49617528915405273 seconds.
Sliced Prediction time is: 504.31 ms
*****************************************
File Name 0000074_07850_d_0000015
Initial Prediction time is: 38.08 ms
Object Density: 53
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  GREEDYNMM
Performing prediction on 28 slices.
Slicing performed in 0.011246442794799805 seconds.
Prediction performed in 0.5319042205810547 seconds.
Sliced Prediction time is: 540.74 ms
*****************************

In [10]:
#USING Adaptive-Optimized-NMS-IoU METHOD (GreedyNMM)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp190/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.79s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.218
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.157
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.159
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.119
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.931
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.072
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.053
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [8]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 512
result_preds_adapt_nms_iou = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000074_03738_d_0000007
Initial Prediction time is: 36.92 ms
Object Density: 52
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  NMS
Performing prediction on 28 slices.
Original Prediction Count 408
Final Bounding Box Count (NMS):  69
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  2
Slicing performed in 0.01049947738647461 seconds.
Prediction performed in 0.49143505096435547 seconds.
Sliced Prediction time is: 499.66 ms
*****************************************
File Name 0000074_07850_d_0000015
Initial Prediction time is: 38.65 ms
Object Density: 53
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  NMS
Performing p

In [9]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp184/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.77s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.159
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.224
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.171
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.182
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.930
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.092
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.060
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [14]:
# Example usage: Adaptive-Optimized-NMS-ALL (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 512
result_preds_adapt_nms_iou = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000074_03738_d_0000007
Initial Prediction time is: 42.76 ms
Object Density: 52
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.
Original Prediction Count 408
Intial Min Area:  1024
Confidence Scores:  0.3
Min area Threshold:  1024
Adaptive Filtered Prediction:  144
Final Bounding Box Count (OptNMS): 35
Adaptive Filtered Prediction:  12
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Slicing performed in 0.012201070785522461 seconds.
Prediction performed in 0.48697447776794434 seconds.
Sliced Prediction time is: 495.79 ms
*****************************************
File Name 0000074_

In [15]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp185/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.46s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.116
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.146
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.109
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.042
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.282
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.022
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [9]:
result_predict_optnms_10_only_area = predict(source='./test_visdrone_data',
                         dataset_json_path = './subset_visdrone_test_data_10.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         postprocess_min_area =  16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Image Name: 0000074_03738_d_0000007
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Original Prediction Count 548
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  511
Final Bounding Box Count (OptNMS): 92
Adaptive Filtered Prediction:  19
Final Bounding Box Count (OptNMS): 10
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Prediction time is: 539.62 ms


...

Original Prediction Count 638
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  535
Final Bounding Box Count (OptNMS): 122
Adaptive Filtered Prediction:  17
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  20
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  54
Final Bounding Box Count (OptNMS): 21
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 4
Prediction time is: 475.53 ms


Performing inference on images:  90%|█████████ | 9/10 [00:06<00:00,  1.42it/s]

Image Name: 0000074_08777_d_0000017
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.


Performing inference on images: 100%|██████████| 10/10 [00:07<00:00,  1.41it/s]

Original Prediction Count 174
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  34
Final Bounding Box Count (OptNMS): 27
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  71
Final Bounding Box Count (OptNMS): 25
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  35
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  12
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 3
Prediction time is: 451.68 ms
Prediction results are successfully exported to runs/predict/exp217
Model loaded in 0.30100560188293457 seconds.
Slicing performed in 0.023608684539794922 seconds.
Prediction performed in 4.975682735443115 seconds.
Exporting performed in 0.0735166072845459 seconds.


In [10]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp217/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.05s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.150
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.226
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.193
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.938
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.094
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [13]:
result_predict_optnms_10 = predict(source='./test_visdrone_data',
                         dataset_json_path = './subset_visdrone_test_data_10.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         postprocess_min_area =  16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Image Name: 0000074_03738_d_0000007
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Original Prediction Count 548
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  511
Final Bounding Box Count (OptNMS): 92
Adaptive Filtered Prediction:  19
Final Bounding Box Count (OptNMS): 10
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Prediction time is: 532.62 ms


...

Original Prediction Count 638
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  535
Final Bounding Box Count (OptNMS): 122
Adaptive Filtered Prediction:  17
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  20
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  54
Final Bounding Box Count (OptNMS): 21
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 4
Prediction time is: 475.29 ms


Performing inference on images:  90%|█████████ | 9/10 [00:06<00:00,  1.44it/s]

Image Name: 0000074_08777_d_0000017
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.


Performing inference on images: 100%|██████████| 10/10 [00:06<00:00,  1.44it/s]

Original Prediction Count 174
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  34
Final Bounding Box Count (OptNMS): 27
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  71
Final Bounding Box Count (OptNMS): 25
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  35
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  12
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 3
Prediction time is: 452.67 ms
Prediction results are successfully exported to runs/predict/exp219
Model loaded in 0.03347349166870117 seconds.
Slicing performed in 0.024357318878173828 seconds.
Prediction performed in 4.826333522796631 seconds.
Exporting performed in 0.07343435287475586 seconds.


In [14]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp219/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.06s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.150
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.226
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.193
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.938
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.094
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [16]:
result_predict_optnms_only_area_10 = predict(source='./test_visdrone_data',
                         dataset_json_path = './subset_visdrone_test_data_10.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         postprocess_min_area =  1024,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Image Name: 0000074_03738_d_0000007
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.


Performing inference on images:  10%|█         | 1/10 [00:00<00:06,  1.37it/s]

Original Prediction Count 548
Intial Min Area:  1024
Confidence Scores:  0.3
Min area Threshold:  1024
Adaptive Filtered Prediction:  176
Final Bounding Box Count (OptNMS): 37
Adaptive Filtered Prediction:  19
Final Bounding Box Count (OptNMS): 10
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Prediction time is: 529.96 ms


...

POST PROCESS:  OptNMS
Performing prediction on 28 slices.


Performing inference on images:  90%|█████████ | 9/10 [00:06<00:00,  1.48it/s]

Original Prediction Count 638
Intial Min Area:  1024
Confidence Scores:  0.3
Min area Threshold:  1024
Adaptive Filtered Prediction:  247
Final Bounding Box Count (OptNMS): 59
Adaptive Filtered Prediction:  17
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 3
Prediction time is: 474.70 ms
Image Name: 0000074_08777_d_0000017
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.


Performing inference on images: 100%|██████████| 10/10 [00:06<00:00,  1.47it/s]

Original Prediction Count 174
Intial Min Area:  1024
Confidence Scores:  0.3
Min area Threshold:  1024
Adaptive Filtered Prediction:  34
Final Bounding Box Count (OptNMS): 27
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  58
Final Bounding Box Count (OptNMS): 19
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  35
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  12
Final Bounding Box Count (OptNMS): 6
Prediction time is: 461.60 ms
Prediction results are successfully exported to runs/predict/exp211
Model loaded in 0.3007638454437256 seconds.
Slicing performed in 0.023446083068847656 seconds.
Prediction performed in 4.833108425140381 seconds.
Exporting performed in 0.07733917236328125 seconds.


In [17]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp211/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.51s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.107
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.133
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.118
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.120
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.025
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.282
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.067
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.011
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [18]:
result_predict_nms_only_area_10 = predict(source='./test_visdrone_data',
                         dataset_json_path = './subset_visdrone_test_data_10.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area =  1024,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Image Name: 0000074_03738_d_0000007
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  NMS
Performing prediction on 28 slices.


Performing inference on images:   0%|          | 0/10 [00:00<?, ?it/s]

Original Prediction Count 548
Final Bounding Box Count (NMS):  76
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  2
Prediction time is: 532.05 ms


...

Original Prediction Count 638
Final Bounding Box Count (NMS):  106
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  21
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  4
Prediction time is: 477.88 ms


Performing inference on images:  90%|█████████ | 9/10 [00:06<00:00,  1.45it/s]

Image Name: 0000074_08777_d_0000017
Image Size:  (1920, 1080)
Sliced Boxes Count: 28
POST PROCESS:  NMS
Performing prediction on 28 slices.


Performing inference on images: 100%|██████████| 10/10 [00:06<00:00,  1.46it/s]

Original Prediction Count 174
Final Bounding Box Count (NMS):  27
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  25
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  3
Prediction time is: 453.59 ms
Prediction results are successfully exported to runs/predict/exp212
Model loaded in 0.11548566818237305 seconds.
Slicing performed in 0.022222518920898438 seconds.
Prediction performed in 4.826605319976807 seconds.
Exporting performed in 0.07347989082336426 seconds.


In [19]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './runs/predict/exp212/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.89s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.149
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.224
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.930
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [15]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 512
result_preds_adapt_opt_nms_iou_only_area = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000074_03738_d_0000007
Initial Prediction time is: 103.30 ms
Object Density: 52
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.
Original Prediction Count 408
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  382
Final Bounding Box Count (OptNMS): 81
Adaptive Filtered Prediction:  12
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Slicing performed in 0.010822534561157227 seconds.
Prediction performed in 0.49153757095336914 seconds.
Sliced Prediction time is: 500.52 ms
*****************************************
File Name 0000074_07

In [16]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp186/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.87s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.160
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.225
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.171
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.184
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.123
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.938
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.060
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [23]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 512
result_preds_adapt_opt_nms_iou_only_area = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000074_03738_d_0000007
Intial Prediction time is: 45.19 ms
Object Density: 52
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  OptNMS
Performing prediction on 28 slices.
Original Prediction Count 408
Intial Min Area:  1024
Confidence Scores:  0.3
Min area Threshold:  1024
Adaptive Filtered Prediction:  144
Final Bounding Box Count (OptNMS): 35
Adaptive Filtered Prediction:  12
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Slicing performed in 0.012244224548339844 seconds.
Prediction performed in 0.6347575187683105 seconds.
Sliced Prediction time is: 643.97 ms
*****************************************
File Name 0000074_07

In [24]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp167/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.46s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.116
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.146
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.109
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.042
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.282
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.022
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [28]:
# Example usage: Adaptive-Optimized-NMS (OptNMS)
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
slice_size = 512
result_preds_adapt_nms_iou_only_area = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000074_03738_d_0000007
Intial Prediction time is: 44.65 ms
Object Density: 52
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  NMS
Performing prediction on 28 slices.
Original Prediction Count 408
Final Bounding Box Count (NMS):  69
Final Bounding Box Count (NMS):  8
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  2
Slicing performed in 0.011269330978393555 seconds.
Prediction performed in 0.5013484954833984 seconds.
Sliced Prediction time is: 510.23 ms
*****************************************
File Name 0000074_07850_d_0000015
Intial Prediction time is: 38.67 ms
Object Density: 53
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  NMS
Performing pre

In [29]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_data_10.json' --result_json_path './sliced_predictions/exp169/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.77s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.159
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.224
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.171
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.182
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.121
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.930
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.092
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.060
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

#### **Subset of 15 images**

In [16]:
result_predict_optnms_10_no_iou_vary = predict(source='./test_vis_data',
                         dataset_json_path = './subset_vis_test_data_15.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "OptNMS",
                         postprocess_min_area =  16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 0000207_00300_d_0000004
Image Size:  (960, 540)
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Original Prediction Count 267
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  12
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  157
Final Bounding Box Count (OptNMS): 37
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 3
Prediction time is: 174.20 ms


...

Image Name: 0000054_00786_d_0000001
Image Size:  (960, 540)
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.
Original Prediction Count 447
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  353
Final Bounding Box Count (OptNMS): 72
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  9
Final Bounding Box Count (OptNMS): 3
Prediction time is: 122.78 ms
Image Name: 0000192_00522_d_0000213
Image Size:  (960, 540)
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:03<00:00,  4.79it/s]

Original Prediction Count 238
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 8
Adaptive Filtered Prediction:  98
Final Bounding Box Count (OptNMS): 23
Adaptive Filtered Prediction:  18
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  51
Final Bounding Box Count (OptNMS): 14
Prediction time is: 113.82 ms
Image Name: 0000078_06777_d_0000020
Image Size:  (1360, 765)
Sliced Boxes Count: 10
POST PROCESS:  OptNMS
Performing prediction on 10 slices.


Performing inference on images: 100%|██████████| 15/15 [00:04<00:00,  3.62it/s]

Original Prediction Count 16
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Prediction time is: 163.86 ms
Prediction results are successfully exported to runs/predict/exp223
Model loaded in 0.032938480377197266 seconds.
Slicing performed in 0.013784408569335938 seconds.
Prediction performed in 2.754720449447632 seconds.
Exporting performed in 0.042464494705200195 seconds.


In [17]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './runs/predict/exp223/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.51s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.109
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.179
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.124
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.185
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.192
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.113
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [12]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS-ALL) - min-area-16
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 512
result_preds_adapt_nms = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000207_00300_d_0000004
Initial Prediction time is: 15.78 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.
Original Prediction Count 213
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  158
Final Bounding Box Count (OptNMS): 41
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNM

In [14]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp200/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.47s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.206
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.337
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.231
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.310
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.483
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.170
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.339
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [17]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS-ALL) - min-area-16
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 512
result_preds_adapt_nmm = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000207_00300_d_0000004
Initial Prediction time is: 17.82 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 6
POST PROCESS:  GREEDYNMM
Performing prediction on 6 slices.
Slicing performed in 0.0030927658081054688 seconds.
Prediction performed in 0.128631591796875 seconds.
Sliced Prediction time is: 131.16 ms
*****************************************
File Name 0000074_07850_d_0000015
Initial Prediction time is: 37.65 ms
Object Density: 53
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  GREEDYNMM
Performing prediction on 28 slices.
Slicing performed in 0.010558843612670898 seconds.
Prediction performed in 0.5306680202484131 seconds.
Sliced Prediction time is: 540.37 ms
******************************

In [18]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS-ALL)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp193/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.52s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.341
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.303
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.497
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.395
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.153
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.326
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [14]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS-ALL) - min-area-16
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 512
result_preds_adapt_nmm = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000207_00300_d_0000004
Initial Prediction time is: 17.60 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 6
POST PROCESS:  NMM
Performing prediction on 6 slices.
Slicing performed in 0.0029909610748291016 seconds.
Prediction performed in 0.14062023162841797 seconds.
Sliced Prediction time is: 143.16 ms
*****************************************
File Name 0000074_07850_d_0000015
Initial Prediction time is: 38.19 ms
Object Density: 53
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Count: 28
POST PROCESS:  NMM
Performing prediction on 28 slices.
Slicing performed in 0.012198925018310547 seconds.
Prediction performed in 0.5969288349151611 seconds.
Sliced Prediction time is: 605.87 ms
****************************************

In [15]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS-ALL)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp192/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.49s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.188
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.338
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.301
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.495
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.395
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.152
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.325
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [11]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS-ALL) - min-area-1024
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 512
result_preds_adapt_nms_opt_iou_all = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000207_00300_d_0000004
Initial Prediction time is: 15.33 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.
Original Prediction Count 213
Intial Min Area:  1024
Confidence Scores:  0.3
Min area Threshold:  1024
Adaptive Filtered Prediction:  9
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 2
Slicing performed in 0.0029633045196533203 seconds.
Prediction performed in 0.11618185043334961 seconds.
Sliced Prediction time is: 118.73 ms
*****************************************
File Name 0000074_07850_d_0000015
Initial Prediction time is: 39.24 ms
Object Density: 53
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes

In [12]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS-ALL)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp180/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.27s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.058
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.087
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.073
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.115
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.046
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.078
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [30]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-NMS)
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 512
result_preds_adapt_nms_iou_only_area = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000207_00300_d_0000004
Intial Prediction time is: 18.77 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 6
POST PROCESS:  NMS
Performing prediction on 6 slices.
Original Prediction Count 213
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  44
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  3
Slicing performed in 0.0031473636627197266 seconds.
Prediction performed in 0.12394595146179199 seconds.
Sliced Prediction time is: 126.63 ms
*****************************************
File Name 0000074_07850_d_0000015
Intial Prediction time is: 40.87 ms
Object Density: 53
********* Slice Parameters ***********
Slic

In [31]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-NMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp170/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.51s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.211
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.348
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.233
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.321
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.490
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.175
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [33]:
result_predict_nms_only_area_15 = predict(source='./test_vis_data',
                         dataset_json_path = './subset_vis_test_data_15.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.25,
                         overlap_width_ratio = 0.25,
                         postprocess_type = "NMS",
                         postprocess_min_area =  1024,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 0000207_00300_d_0000004
Image Size:  (960, 540)
Sliced Boxes Count: 6
POST PROCESS:  NMS
Performing prediction on 6 slices.


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Original Prediction Count 265
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  53
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  5
Prediction time is: 174.96 ms


...

Image Name: 0000054_00786_d_0000001
Image Size:  (960, 540)
Sliced Boxes Count: 6
POST PROCESS:  NMS
Performing prediction on 6 slices.
Original Prediction Count 432
Final Bounding Box Count (NMS):  77
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  5
Prediction time is: 124.85 ms
Image Name: 0000192_00522_d_0000213
Image Size:  (960, 540)
Sliced Boxes Count: 6
POST PROCESS:  NMS
Performing prediction on 6 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:03<00:00,  5.03it/s]

Original Prediction Count 245
Final Bounding Box Count (NMS):  3
Final Bounding Box Count (NMS):  9
Final Bounding Box Count (NMS):  25
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  16
Prediction time is: 115.20 ms
Image Name: 0000078_06777_d_0000020
Image Size:  (1360, 765)
Sliced Boxes Count: 8
POST PROCESS:  NMS
Performing prediction on 8 slices.


Performing inference on images: 100%|██████████| 15/15 [00:03<00:00,  4.12it/s]

Original Prediction Count 13
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Prediction time is: 140.15 ms
Prediction results are successfully exported to runs/predict/exp214
Model loaded in 0.03020501136779785 seconds.
Slicing performed in 0.01378941535949707 seconds.
Prediction performed in 2.247627019882202 seconds.
Exporting performed in 0.04330778121948242 seconds.


In [34]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-NMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './runs/predict/exp214/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.55s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.207
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.351
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.224
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.329
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.449
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.308
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.178
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.302
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [36]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS)
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 512
result_preds_adapt_nms_opt_iou_only_area = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000207_00300_d_0000004
Intial Prediction time is: 18.44 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.
Original Prediction Count 213
Intial Min Area:  1024
Confidence Scores:  0.3
Min area Threshold:  1024
Adaptive Filtered Prediction:  9
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 2
Slicing performed in 0.003306150436401367 seconds.
Prediction performed in 0.11951136589050293 seconds.
Sliced Prediction time is: 122.35 ms
*****************************************
File Name 0000074_07850_d_0000015
Intial Prediction time is: 41.62 ms
Object Density: 53
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes Co

In [37]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-NMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp171/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.27s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.058
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.087
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.073
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.115
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.046
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.078
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [8]:
# Example usage: Adaptive-Optimized-NMS-Confidence (Adaptive-OptNMS)
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 512
result_preds_adapt_nms_opt_iou_only_area = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000207_00300_d_0000004
Intial Prediction time is: 15.92 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.
Original Prediction Count 213
Intial Min Area:  1024
Confidence Scores:  0.3
Min area Threshold:  1024
Adaptive Filtered Prediction:  9
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 2
Slicing performed in 0.0029506683349609375 seconds.
Prediction performed in 0.11695647239685059 seconds.
Sliced Prediction time is: 119.60 ms
*****************************************
File Name 0000074_07850_d_0000015
Intial Prediction time is: 39.56 ms
Object Density: 53
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes C

In [9]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp174/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.27s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.058
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.087
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.073
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.115
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.046
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.078
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [11]:
# Example usage: Adaptive-Optimized-NMS-Confidence (Adaptive-OptNMS)
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 512
result_preds_adapt_nms_opt_iou_only_area = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000207_00300_d_0000004
Intial Prediction time is: 17.31 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.
Original Prediction Count 213
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  158
Final Bounding Box Count (OptNMS): 49
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS

In [12]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp175/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.57s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.210
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.346
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.233
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.320
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.490
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [15]:
# Example usage: Adaptive-Optimized-NMS-Confidence (Adaptive-NMS) - min-area-16
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 512
result_preds_adapt_nms_opt_iou_only_area = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000207_00300_d_0000004
Intial Prediction time is: 17.37 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 6
POST PROCESS:  NMS
Performing prediction on 6 slices.
Original Prediction Count 213
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  44
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  3
Slicing performed in 0.003190755844116211 seconds.
Prediction performed in 0.12372398376464844 seconds.
Sliced Prediction time is: 126.74 ms
*****************************************
File Name 0000074_07850_d_0000015
Intial Prediction time is: 41.08 ms
Object Density: 53
********* Slice Parameters ***********
Slice

In [16]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp176/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.51s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.211
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.348
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.233
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.321
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.490
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.175
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [8]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS) - min-area-16
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 512
result_preds_adapt_nms_opt_iou_only_area = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000207_00300_d_0000004
Intial Prediction time is: 15.03 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.
Original Prediction Count 213
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  158
Final Bounding Box Count (OptNMS): 49
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS

In [9]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp178/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.57s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.210
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.346
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.233
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.320
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.490
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [15]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS) - min-area-16
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 512
result_preds_adapt_nms_opt_iou_ALL = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000207_00300_d_0000004
Initial Prediction time is: 15.88 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.
Original Prediction Count 213
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  158
Final Bounding Box Count (OptNMS): 49
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNM

In [16]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp182/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.57s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.210
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.346
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.233
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.320
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.490
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [11]:
result_predict_nms_only_area_15 = predict(source='./test_vis_data',
                         dataset_json_path = './subset_vis_test_data_15.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 512,
                         slice_width = 512,
                         overlap_height_ratio = 0.25,
                         overlap_width_ratio = 0.25,
                         postprocess_type = "OptNMS",
                         postprocess_min_area =  16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: OptNMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Image Name: 0000207_00300_d_0000004
Image Size:  (960, 540)
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.


Performing inference on images:   0%|          | 0/15 [00:00<?, ?it/s]

Original Prediction Count 265
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  14
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  194
Final Bounding Box Count (OptNMS): 60
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  14
Final Bounding Box Count (OptNMS): 9
Adaptive Filtered Prediction:  7
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 5
Prediction time is: 174.97 ms


...

Image Name: 0000054_00786_d_0000001
Image Size:  (960, 540)
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.
Original Prediction Count 432
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  407
Final Bounding Box Count (OptNMS): 89
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  8
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  12
Final Bounding Box Count (OptNMS): 5
Prediction time is: 123.59 ms
Image Name: 0000192_00522_d_0000213
Image Size:  (960, 540)
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.


Performing inference on images:  93%|█████████▎| 14/15 [00:03<00:00,  5.03it/s]

Original Prediction Count 245
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 3
Adaptive Filtered Prediction:  35
Final Bounding Box Count (OptNMS): 10
Adaptive Filtered Prediction:  116
Final Bounding Box Count (OptNMS): 26
Adaptive Filtered Prediction:  18
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  12
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  58
Final Bounding Box Count (OptNMS): 16
Prediction time is: 113.73 ms
Image Name: 0000078_06777_d_0000020
Image Size:  (1360, 765)
Sliced Boxes Count: 8
POST PROCESS:  OptNMS
Performing prediction on 8 slices.


Performing inference on images: 100%|██████████| 15/15 [00:03<00:00,  4.11it/s]

Original Prediction Count 13
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  10
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Prediction time is: 134.10 ms
Prediction results are successfully exported to runs/predict/exp218
Model loaded in 0.032855987548828125 seconds.
Slicing performed in 0.014453887939453125 seconds.
Prediction performed in 2.218278169631958 seconds.
Exporting performed in 0.04270601272583008 seconds.


In [12]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './runs/predict/exp218/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.63s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.342
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.221
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.329
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.434
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.308
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.178
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.294
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [17]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS) - min-area-16
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 512
result_preds_adapt_nms_opt_iou_only_area = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000207_00300_d_0000004
Initial Prediction time is: 19.68 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.
Original Prediction Count 213
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  13
Final Bounding Box Count (OptNMS): 6
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  158
Final Bounding Box Count (OptNMS): 49
Adaptive Filtered Prediction:  3
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  11
Final Bounding Box Count (OptNMS): 7
Adaptive Filtered Prediction:  5
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNM

In [18]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS) --> min_area - 16
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp187/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.57s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.210
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.346
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.233
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.320
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.490
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [20]:
# Example usage: Adaptive-Optimized-NMS (Adaptive-OptNMS) - min-area-1024
source_folder = './test_vis_data/images'
json_path = "./subset_vis_test_data_15.json"
slice_size = 512
result_preds_adapt_nms_opt_iou_all = predict_sliced_images(source_folder, json_path, detection_model, slice_size)

*****************************************
File Name 0000207_00300_d_0000004
Initial Prediction time is: 19.77 ms
Object Density: 25
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.25
Overlap Height Ratio:  0.25
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Performing prediction on 6 slices.
Original Prediction Count 213
Intial Min Area:  1024
Confidence Scores:  0.3
Min area Threshold:  1024
Adaptive Filtered Prediction:  9
Final Bounding Box Count (OptNMS): 4
Adaptive Filtered Prediction:  4
Final Bounding Box Count (OptNMS): 2
Slicing performed in 0.0032682418823242188 seconds.
Prediction performed in 0.11909866333007812 seconds.
Sliced Prediction time is: 121.85 ms
*****************************************
File Name 0000074_07850_d_0000015
Initial Prediction time is: 40.49 ms
Object Density: 53
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.5
Overlap Height Ratio:  0.5
Sliced Boxes

In [21]:
#USING Adaptive-Optimized-NMS-IoU METHOD (Adaptive-OptNMS) --> min_area - 1024
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_15.json' --result_json_path './sliced_predictions/exp188/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.27s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.058
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.087
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.065
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.073
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.115
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.046
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.078
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

#### Test

In [3]:
import os
import time
import json
import numpy as np
from PIL import Image
from multiprocessing import Pool, cpu_count
from pathlib import Path
from sahi.predict import get_prediction, get_sliced_prediction
from sahi.utils.file import save_json
from sahi.prediction import visualize_object_predictions


def get_dynamic_slice_parameters(object_density, base_slice_size):
    if object_density > 30:
        return base_slice_size // 2, base_slice_size // 2, 0.4, 0.4
    elif object_density > 15:
        return base_slice_size // 2, base_slice_size // 2, 0.3, 0.3
    elif object_density > 10:
        return base_slice_size, base_slice_size, 0.2, 0.2
    else:
        return None


def run_single_image(args):
    image_path, model, slice_size, save_dir, image_id, visualize = args
    image = Image.open(image_path).convert("RGB")
    filename = os.path.basename(image_path)
    filename_wo_ext = os.path.splitext(filename)[0]

    image_as_pil = image.copy()
    start_time = time.time()
    base_prediction = get_prediction(image_path, model)
    base_time = time.time() - start_time
    object_density = len(base_prediction.object_prediction_list)

    params = get_dynamic_slice_parameters(object_density, slice_size)

    if params is None:
        final_result = base_prediction
    else:
        slice_width, slice_height, overlap_w, overlap_h = params
        final_result = get_sliced_prediction(
            image_path,
            model,
            slice_height=slice_height,
            slice_width=slice_width,
            overlap_height_ratio=overlap_h,
            overlap_width_ratio=overlap_w,
            postprocess_type="NMS",
            postprocess_match_metric="IOU",
            postprocess_match_threshold=0.5,
            postprocess_min_area=32,
            verbose=0
        )

    if visualize:
        visualize_object_predictions(
            np.ascontiguousarray(image_as_pil),
            object_prediction_list=final_result.object_prediction_list,
            rect_th=2,
            text_size=0.5,
            text_th=1,
            hide_labels=False,
            hide_conf=False,
            output_dir=save_dir,
            file_name=filename_wo_ext,
            export_format="png",
        )

    # Convert to COCO and return
    coco_preds = final_result.to_coco_predictions(image_id=image_id)
    return coco_preds, len(final_result.object_prediction_list), base_time


def batch_adaptive_sliced_prediction(
    input_folder,
    model,
    slice_size=512,
    save_dir="sliced_predictions",
    visualize=True,
    dataset_json_path=None
):
    os.makedirs(save_dir, exist_ok=True)

    # Load COCO annotation (if provided)
    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)
    else:
        data = {}

    all_images = [
        os.path.join(input_folder, f)
        for f in os.listdir(input_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    tasks = []
    for idx, image_path in enumerate(all_images):
        tasks.append((image_path, model, slice_size, save_dir, idx + 1, visualize))

    print(f"\n🚀 Starting batch prediction on {len(tasks)} images using {min(cpu_count(), 4)} workers...\n")

    total_detections = 0
    total_time = 0.0
    coco_all_predictions = []

    with Pool(processes=min(cpu_count(), 4)) as pool:
        for coco_preds, count, timing in pool.imap_unordered(run_single_image, tasks):
            total_detections += count
            total_time += timing
            coco_all_predictions.extend(coco_preds)

    # Save COCO JSON
    result_json_path = os.path.join(save_dir, "result.json")
    save_json(coco_all_predictions, result_json_path)

    print(f"\n✅ Completed batch slicing:")
    print(f"   Total images processed: {len(tasks)}")
    print(f"   Total detections: {total_detections}")
    print(f"   Total time: {total_time:.2f} s")
    print(f"   COCO results saved to: {result_json_path}")


In [ ]:
# Example usage: Adaptive-Optimized-NMS
source_folder = './test_visdrone_data/images'
json_path = "./subset_visdrone_test_data_10.json"
batch_adaptive_sliced_prediction(
    input_folder=source_folder,
    model=detection_model,
    slice_size=512,
    save_dir="./output_folder",
    visualize=True,
    dataset_json_path=json_path  # or path to ground truth COCO JSON
)
